In [1]:
# !pip install pandas-gbq
# !pip install --upgrade google-cloud-bigquery

In [2]:
import requests
import json
from pprint import pprint
import pandas as pd
from pandas.io import gbq
import os
from dotenv import load_dotenv

#### Pull data from data.gov.in

In [3]:
def get_data():
    '''
        A function to fetch data from API (data.gov.in)
    '''
    resource = "/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69"
    api_key = "579b464db66ec23bdd000001d16bebe4295744234023fe46120b3f81"

    url = f"https://api.data.gov.in{resource}?api-key={api_key}&format=json&limit=150"
    headers = {
        "api-key": "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b",
        "format": "json"
    }

    response = requests.get(url, headers=headers)
    data = json.loads(response.content.decode())

    data_df = pd.DataFrame(data=data["records"])
    data_df = data_df[(data_df["pollutant_avg"] != "NA") & (data_df["pollutant_max"] != "NA") & (data_df["pollutant_min"] != "NA")]
    # data_df = data_df.astype({"pollutant_min": "int32", "pollutant_max": "int32", "pollutant_avg": "int32"})
    
    return data_df

In [4]:
data_df = get_data()

#### Push data into BigQuery

In [5]:
project_id = "steel-capsule-372509"
dataset_id = "realtime_aqi"
table_name = "everything"

In [6]:
data_df.to_gbq(
    destination_table=f"{dataset_id}.{table_name}",
    project_id=project_id,
    if_exists="replace", # fail/append/replace
)

100%|██████████| 1/1 [00:00<00:00, 2534.32it/s]


#### Run queries on BigQuery

In [11]:
query = """SELECT city, state, pollutant_id, pollutant_avg
    FROM `steel-capsule-372509.realtime_aqi.everything`
    WHERE pollutant_id IN('PM2.5', 'PM10')
"""

In [12]:
result_df = pd.read_gbq(query, project_id)
result_df

,city,state,pollutant_id,pollutant_avg
0,Amaravati,Andhra_Pradesh,PM2.5,256
1,Amaravati,Andhra_Pradesh,PM10,142
2,Anantapur,Andhra_Pradesh,PM2.5,32
3,Anantapur,Andhra_Pradesh,PM10,54
4,Rajamahendravaram,Andhra_Pradesh,PM10,115
5,Tirupati,Andhra_Pradesh,PM2.5,176
6,Tirupati,Andhra_Pradesh,PM10,105
7,Tirupati,Andhra_Pradesh,PM2.5,50
8,Tirupati,Andhra_Pradesh,PM10,73
9,Visakhapatnam,Andhra_Pradesh,PM2.5,193


In [9]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

In [10]:
# from google.cloud import bigquery

# client = bigquery.Client(project=project_id)
# job = client.query(query=query)

# result = job.result()
# result